# Mount drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install libraries

In [2]:
!pip install -q python-dotenv langchain langchain-openai openai langchain-community langchain_objectbox langchainhub gradio

# Parameters

In [49]:
selected_model = "gpt-3.5-turbo"
temperature = 0.4
top_p = 0.8

# Api key

In [3]:
import os
from dotenv import load_dotenv
load_dotenv("drive/MyDrive/Colab/.env")

True

# Load Data

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader

txt_file_path = 'drive/MyDrive/Colab/docs/houdini_dataset.txt'
loader = TextLoader(file_path=txt_file_path, encoding="utf-8")
data = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(data)

# Embeddings

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_objectbox.vectorstores import ObjectBox
vector = ObjectBox.from_documents(documents, OpenAIEmbeddings(), embedding_dimensions=768)

# RAG Pipeline

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [52]:
#llm = ChatOpenAI(model=selected_model)
llm = ChatOpenAI(model=selected_model, temperature=temperature, model_kwargs={'top_p': top_p})
prompt = hub.pull("rlm/rag-prompt")

## Overwrite prompt template

In [53]:
prompt.messages[0].prompt.template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Firstly provide a short summarized answer. Afterwards provide a detailed answer.\nQuestion: {question} \nContext: {context} \nAnswer:"""

# QA chain

In [54]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vector.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [55]:
question = "How to create velocity field from geometry in houdini?"
result = qa_chain.invoke({"query": question})
result

{'query': 'How to create velocity field from geometry in houdini?',
 'result': "To create a velocity field from geometry in Houdini, you can use the 'vdb from polygons' node to efficiently convert meshes into sdf or fog, and create a velocity field if you have @v on your mesh. If you don't have a watertight mesh, you may need to create manifold geometry with extrudes. Another method involves using a volume wrangle to create a generic fog of density and then do an attribute transfer of @v from the curve to @vel of the volume.\n\nDetailed Answer:\nTo create a velocity field from geometry in Houdini, you can follow these steps:\n\n1. Use the 'vdb from polygons' node to convert meshes into sdf or fog. This node can also create a velocity field if your mesh has @v attribute.\n2. If you have a mesh like a pig, give it point normals and create a swirled @v in a wrangle with @v = cross(@N,{0,1,0}).\n3. Use the 'vdb from polygons' node to make a density field and create a @vel field from @v.\n4

# Adding UI

In [56]:
import gradio as gr

# Initialize a global list to keep track of conversation history
conversation_history = []


def talk(message, history):
    # The 'history' parameter includes past interactions
    # We'll extract the question and answer texts to build a full context
    context = " ".join([f"Q: {exchange['input']} A: {exchange['output']}" for exchange in history])
    full_question = f"{context} Q: {message}"

    # Assume 'qa_chain' is your model that processes the full question
    result = qa_chain.invoke({"query": full_question})
    answer = result['result']

    return answer

# Define the chatbot interface layout and properties
TITLE = "Houdini Chatbot"
DESCRIPTION = f"{selected_model} with RAG to answer Houdini-related questions based on tokeru.com/cgwiki"
chatbot = gr.Chatbot(
    show_label=True,
    show_share_button=True,
    show_copy_button=True,
    likeable=True,
    layout="bubble",
    bubble_full_width=True,
    height=700,
)

# Create the ChatInterface
demo = gr.ChatInterface(
    fn=talk,
    chatbot=chatbot,
    theme="Soft",
    title=TITLE,
    description=DESCRIPTION
)

In [ ]:
# Launch the Gradio interface
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://007958e15839f17257.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
